In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
X = pd.read_csv('train_dummies.csv')
y = pd.read_csv('train.csv')['SalePrice']

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.svm import SVR
base_model = SVR()

In [7]:
base_model.fit(X_train, y_train)

SVR()

In [8]:
base_preds = base_model.predict(X_test)

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [10]:
mae = mean_absolute_error(y_test, base_preds)
mae

54962.067737780344

In [12]:
rmse = np.sqrt(mean_squared_error(y_test, base_preds))
rmse

80725.55858424804

In [13]:
from sklearn.model_selection import GridSearchCV

In [14]:
param_grid = {'C': [0.001,0.01,0.1,0.5,1], 'kernel': ['linear','rbf','poly'], 'gamma': ['scale', 'auto'], 
             'degree': [2,3,4], 'epsilon': [0,0.01,0.1,0.5,1,2]}

In [15]:
svr = SVR()

In [16]:
grid = GridSearchCV(svr, param_grid)

In [17]:
grid.fit(X_train, y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1], 'degree': [2, 3, 4],
                         'epsilon': [0, 0.01, 0.1, 0.5, 1, 2],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']})

In [18]:
svr_preds = grid.predict(X_test)

In [21]:
mae = mean_absolute_error(y_test, svr_preds)
mae

45471.67604320109

In [22]:
rmse = np.sqrt(mean_squared_error(y_test, svr_preds))
rmse

71459.38384766823

In [23]:
final_test = pd.read_csv('test_dummies.csv')

In [27]:
final_test = scaler.transform(final_test)

In [28]:
final_preds = grid.predict(final_test)

In [29]:
final_preds

array([110402.92261792, 123238.89168535, 146519.63787693, ...,
       119490.59941977,  89982.10610647, 140418.82005824])

In [30]:
submission = pd.concat([pd.read_csv('test.csv')['Id'],pd.DataFrame(final_preds)], axis=1)
submission.columns = ['Id', 'SalePrice']

In [31]:
submission.to_csv('SVRsubmission.csv', index=False)